# Python Cookbook: Chapter 6 - Data Encoding & Processing
Code samples by chapter: https://github.com/dabeaz/python-cookbook/tree/master/src/

## Gotchas to remember

...

## 6.1 - Reading & Writing CVS data


In [125]:
import csv
import requests

# Write example csv file locally
csv_url = 'https://raw.githubusercontent.com/dabeaz/python-cookbook/master/src/6/reading_and_writing_csv_data/stocks.csv'
response = requests.get(csv_url)
stocks_file = 'working_data/stocks.csv'
with open(stocks_file, 'w') as f:
    f.write(response.text)

# Read it
with open(stocks_file) as f:
    # Read as list of strings
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        # Access by list index
        print(row[0], row[1], row[2])
        # Or print all items in list
        print(', '.join(row))

AA 39.48 6/11/2007
AA, 39.48, 6/11/2007, 9:36am, -0.18, 181800
AIG 71.38 6/11/2007
AIG, 71.38, 6/11/2007, 9:36am, -0.15, 195500
AXP 62.58 6/11/2007
AXP, 62.58, 6/11/2007, 9:36am, -0.46, 935000
BA 98.31 6/11/2007
BA, 98.31, 6/11/2007, 9:36am, +0.12, 104800
C 53.08 6/11/2007
C, 53.08, 6/11/2007, 9:36am, -0.25, 360900
CAT 78.29 6/11/2007
CAT, 78.29, 6/11/2007, 9:36am, -0.23, 225400


In [126]:
# Use namedtuple for readability
from collections import namedtuple

with open(stocks_file) as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    Row = namedtuple('Row', headers)
    for r in f_csv:
        row = Row(*r)     # Note, the namedtuple must be instatiated w/ positional (or keyword) args
        print('Symbol:', row.Symbol, 'Price:', row.Price)

Symbol: AA Price: 39.48
Symbol: AIG Price: 71.38
Symbol: AXP Price: 62.58
Symbol: BA Price: 98.31
Symbol: C Price: 53.08
Symbol: CAT Price: 78.29


In [127]:
# Load as dict

# Save for later writing
dict_rows = []

with open(stocks_file) as f:
    f_csv = csv.DictReader(f)    # Note it assumes first row contains headers if not passed as args
    for row in f_csv:
        dict_rows.append(row)
        print('Symbol:', row['Symbol'], 'Price:', row['Price'])

Symbol: AA Price: 39.48
Symbol: AIG Price: 71.38
Symbol: AXP Price: 62.58
Symbol: BA Price: 98.31
Symbol: C Price: 53.08
Symbol: CAT Price: 78.29


In [128]:
# Write csv file

# Load data first
rows = []
with open(stocks_file) as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        rows.append(row)

# Write data
with open(stocks_file + '.out', 'w') as f:
    f_csv_out = csv.writer(f)
    f_csv_out.writerow(headers)
    f_csv_out.writerows(rows)

# Show results using shell for fun -- See Recipe 13.6
import subprocess
# returns a *byte string*
out = subprocess.check_output(['cat', stocks_file + '.out'])
print(out.decode('utf-8'))

Symbol,Price,Date,Time,Change,Volume
AA,39.48,6/11/2007,9:36am,-0.18,181800
AIG,71.38,6/11/2007,9:36am,-0.15,195500
AXP,62.58,6/11/2007,9:36am,-0.46,935000
BA,98.31,6/11/2007,9:36am,+0.12,104800
C,53.08,6/11/2007,9:36am,-0.25,360900
CAT,78.29,6/11/2007,9:36am,-0.23,225400



In [129]:
# Write from sequence of dicts
with open(stocks_file + '.dict.out', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(dict_rows)

# Show results using shell for fun -- See Recipe 13.6
import subprocess
# returns a *byte string*
out = subprocess.check_output(['cat', stocks_file + '.dict.out'])
print(out.decode('utf-8'))

Symbol,Price,Date,Time,Change,Volume
AA,39.48,6/11/2007,9:36am,-0.18,181800
AIG,71.38,6/11/2007,9:36am,-0.15,195500
AXP,62.58,6/11/2007,9:36am,-0.46,935000
BA,98.31,6/11/2007,9:36am,+0.12,104800
C,53.08,6/11/2007,9:36am,-0.25,360900
CAT,78.29,6/11/2007,9:36am,-0.23,225400



### Read cvs w/ pandas 

In [130]:
import pandas as pd

df = pd.read_csv(stocks_file)
df.head()

,Symbol,Price,Date,Time,Change,Volume
0,AA,39.48,6/11/2007,9:36am,-0.18,181800
1,AIG,71.38,6/11/2007,9:36am,-0.15,195500
2,AXP,62.58,6/11/2007,9:36am,-0.46,935000
3,BA,98.31,6/11/2007,9:36am,0.12,104800
4,C,53.08,6/11/2007,9:36am,-0.25,360900


## 6.1 - Reading & Writing JSON data

In [131]:
import json

# Note putting dicts within list is better json structure, esp. for reading into pandas DataFrame
data = [
    {
        'symbol': 'AA',
        'price': 33.48,
        'date': '6/11/2007'
    },
    {
        "symbol": "AIG",
        "price": 71.38,
        "date": "6/11/2007"
    }
]

# Turn python object into string
json_str = json.dumps(data)
print(json_str)

[{"symbol": "AA", "price": 33.48, "date": "6/11/2007"}, {"symbol": "AIG", "price": 71.38, "date": "6/11/2007"}]


In [132]:
# Turn back into python object
data = json.loads(json_str)
print('Object type:', type(data))
print(data)

Object type: <class 'list'>
[{'symbol': 'AA', 'price': 33.48, 'date': '6/11/2007'}, {'symbol': 'AIG', 'price': 71.38, 'date': '6/11/2007'}]


In [133]:
# Write & re-read the data as json
json_file = 'working_data/stocks.json'
with open(json_file, 'w') as f:
    json.dump(data, f)


with open(json_file) as f:
    data_loaded = json.load(f)
    print(data_loaded)

[{'symbol': 'AA', 'price': 33.48, 'date': '6/11/2007'}, {'symbol': 'AIG', 'price': 71.38, 'date': '6/11/2007'}]


In [134]:
# Pretty print a larger json string
from pprint import pprint

dict_rows = []
with open(stocks_file) as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        dict_rows.append(row)

# Doesn't do much.
pprint(json.dumps(dict_rows))

('[{"Symbol": "AA", "Price": "39.48", "Date": "6/11/2007", "Time": "9:36am", '
 '"Change": "-0.18", "Volume": "181800"}, {"Symbol": "AIG", "Price": "71.38", '
 '"Date": "6/11/2007", "Time": "9:36am", "Change": "-0.15", "Volume": '
 '"195500"}, {"Symbol": "AXP", "Price": "62.58", "Date": "6/11/2007", "Time": '
 '"9:36am", "Change": "-0.46", "Volume": "935000"}, {"Symbol": "BA", "Price": '
 '"98.31", "Date": "6/11/2007", "Time": "9:36am", "Change": "+0.12", "Volume": '
 '"104800"}, {"Symbol": "C", "Price": "53.08", "Date": "6/11/2007", "Time": '
 '"9:36am", "Change": "-0.25", "Volume": "360900"}, {"Symbol": "CAT", "Price": '
 '"78.29", "Date": "6/11/2007", "Time": "9:36am", "Change": "-0.23", "Volume": '
 '"225400"}]')


In [138]:
# Also pretty print using indent option; sort_keys option too
print(json.dumps(dict_rows, indent=4, sort_keys=True))

[
    {
        "Change": "-0.18",
        "Date": "6/11/2007",
        "Price": "39.48",
        "Symbol": "AA",
        "Time": "9:36am",
        "Volume": "181800"
    },
    {
        "Change": "-0.15",
        "Date": "6/11/2007",
        "Price": "71.38",
        "Symbol": "AIG",
        "Time": "9:36am",
        "Volume": "195500"
    },
    {
        "Change": "-0.46",
        "Date": "6/11/2007",
        "Price": "62.58",
        "Symbol": "AXP",
        "Time": "9:36am",
        "Volume": "935000"
    },
    {
        "Change": "+0.12",
        "Date": "6/11/2007",
        "Price": "98.31",
        "Symbol": "BA",
        "Time": "9:36am",
        "Volume": "104800"
    },
    {
        "Change": "-0.25",
        "Date": "6/11/2007",
        "Price": "53.08",
        "Symbol": "C",
        "Time": "9:36am",
        "Volume": "360900"
    },
    {
        "Change": "-0.23",
        "Date": "6/11/2007",
        "Price": "78.29",
        "Symbol": "CAT",
        "Time": "9:36am"

### Read json w/ pandas 

In [136]:
# Note: When json string contains a *list* of dicts it can be loaded as DataFrame; numerical index is assumed if none specified
df_json = pd.read_json(json_file)
df_json.head()

,symbol,price,date
0,AA,33.48,2007-06-11
1,AIG,71.38,2007-06-11


In [137]:
# A single dict record in the json can be loaded as a *Series*
data_series = {
        'symbol': 'AA',
        'price': 33.48,
        'date': '6/11/2007'
    }
series = pd.read_json(json.dumps(data_series), typ='series')
series.head()

symbol           AA
price         33.48
date      6/11/2007
dtype: object

### Using object hooks

In [143]:
# object_pairs_hook option

from collections import OrderedDict

json_s = '[{"symbol": "AA", "price": 33.48, "date": "6/11/2007"}, {"symbol": "AIG", "price": 71.38, "date": "6/11/2007"}]'

dict = json.loads(json_s, object_pairs_hook=OrderedDict)
print(dict)

[OrderedDict([('symbol', 'AA'), ('price', 33.48), ('date', '6/11/2007')]), OrderedDict([('symbol', 'AIG'), ('price', 71.38), ('date', '6/11/2007')])]


In [ ]:
# object_hook option


